In [3]:
from contextlib import asynccontextmanager

from fastapi import FastAPI
import google.auth
from google.cloud import aiplatform, storage
import os
import logging

logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)

# TODO - set Configuration as env vars
MODEL_ID = "1372802939441119232"
LOCAL_MODEL_DIR = "./model"
GOOGLE_APPLICATION_CREDENTIALS = "../level-agent-451514-c0-fc48725fe84a.json"

credentials, project_id = google.auth.load_credentials_from_file(GOOGLE_APPLICATION_CREDENTIALS)

aiplatform.init(
    project=project_id,
    location="us-central1",
    credentials=credentials
)

In [4]:
model = aiplatform.Model(model_name="1372802939441119232")

In [13]:


model_uri = model.uri

if not model_uri:
    raise ValueError("Model URI is not available")

if not model_uri.startswith("gs://"):
    raise ValueError(f"Expected gs:// path but got {model_uri}")


In [20]:
bucket_name = model_uri.split("/")[2]
prefix = "/".join(model_uri.split("/")[3:])

# Initialize GCS client
client = storage.Client(credentials=credentials)
bucket = client.bucket(bucket_name)

# List objects to find the .pkl file
blobs = bucket.list_blobs(prefix=prefix)
model_files = [blob for blob in blobs if blob.name.endswith('.pkl')]

if not model_files:
    raise ValueError("No .pkl file found in model artifacts")

for model in model_files:
    if model.name.endswith("model.pkl"):
        model.download_to_filename("model.pkl")

In [29]:
req =     [{
        "MSSubClass": 60,
        "MSZoning": "RL",
        "LotFrontage": 65.0,
        "LotArea": 8450,
        "Street": "Pave",
        "Alley": None,
        "LotShape": "Reg",
        "LandContour": "Lvl",
        "Utilities": "AllPub",
        "LotConfig": "Inside",
        "LandSlope": "Gtl",
        "Neighborhood": "CollgCr",
        "Condition1": "Norm",
        "Condition2": "Norm",
        "BldgType": "1Fam",
        "HouseStyle": "2Story",
        "OverallQual": 7,
        "OverallCond": 5,
        "YearBuilt": 2003,
        "YearRemodAdd": 2003,
        "RoofStyle": "Gable",
        "RoofMatl": "CompShg",
        "Exterior1st": "VinylSd",
        "Exterior2nd": "VinylSd",
        "MasVnrType": "BrkFace",
        "MasVnrArea": 196.0,
        "ExterQual": "Gd",
        "ExterCond": "TA",
        "Foundation": "PConc",
        "BsmtQual": "Gd",
        "BsmtCond": "TA",
        "BsmtExposure": "No",
        "BsmtFinType1": "GLQ",
        "BsmtFinSF1": 706,
        "BsmtFinType2": "Unf",
        "BsmtFinSF2": 0,
        "BsmtUnfSF": 150,
        "TotalBsmtSF": 856,
        "Heating": "GasA",
        "HeatingQC": "Ex",
        "CentralAir": "Y",
        "Electrical": "SBrkr",
        "1stFlrSF": 856,
        "2ndFlrSF": 854,
        "LowQualFinSF": 0,
        "GrLivArea": 1710,
        "BsmtFullBath": 1,
        "BsmtHalfBath": 0,
        "FullBath": 2,
        "HalfBath": 1,
        "BedroomAbvGr": 3,
        "KitchenAbvGr": 1,
        "KitchenQual": "Gd",
        "TotRmsAbvGrd": 8,
        "Functional": "Typ",
        "Fireplaces": 0,
        "FireplaceQu": None,
        "GarageType": "Attchd",
        "GarageYrBlt": 2003.0,
        "GarageFinish": "RFn",
        "GarageCars": 2,
        "GarageArea": 548,
        "GarageQual": "TA",
        "GarageCond": "TA",
        "PavedDrive": "Y",
        "WoodDeckSF": 0,
        "OpenPorchSF": 61,
        "EnclosedPorch": 0,
        "3SsnPorch": 0,
        "ScreenPorch": 0,
        "PoolArea": 0,
        "PoolQC": None,
        "Fence": None,
        "MiscFeature": None,
        "MiscVal": 0,
        "MoSold": 2,
        "YrSold": 2008,
        "SaleType": "WD",
        "SaleCondition": "Normal"
    }]

In [9]:
import pandas as pd

In [30]:
req_df = pd.DataFrame(req)

In [31]:
req_df.head()

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,60,RL,65.0,8450,Pave,None,Reg,Lvl,AllPub,Inside,...,0,0,None,None,None,0,2,2008,WD,Normal


In [ ]:
import joblib
model = joblib.load("model.pkl")

In [33]:
model.predict(req_df)

array([12.14823236])

In [34]:
model = aiplatform.Model.upload(
    display_name="test-from-container",
    artifact_uri=f"gs://house-pricing-tryolabs/artifacts",
    serving_container_image_uri="us-central1-docker.pkg.dev/level-agent-451514-c0/ml-housing-repo/housing-model",
)

Creating Model
Create Model backing LRO: projects/954208583758/locations/us-central1/models/4752754459782676480/operations/6930969354699800576
Model created. Resource name: projects/954208583758/locations/us-central1/models/4752754459782676480@1
To use this Model in another session:
model = aiplatform.Model('projects/954208583758/locations/us-central1/models/4752754459782676480@1')


In [5]:
model_name = "my-first-models"

model_registry = aiplatform.Model.list(filter=f'display_name="{model_name}"')

In [6]:
model_registry

 resource name: projects/954208583758/locations/us-central1/models/1372802939441119232]

In [13]:
# Find existing model (highest version)
existing_model = None
highest_version = -1
for model in model_registry:
    try:
        version_id = int(model.version_id)
        if version_id > highest_version:
            highest_version = version_id
            existing_model = model
    except (ValueError, TypeError):
        logging.warning(f"Invalid version ID for model: {model.resource_name}")
        continue

if not existing_model:
    logging.warning("No valid existing model found.")

In [14]:
existing_rmse = float(existing_model.labels.get("rmse", float("inf")))

In [15]:
existing_rmse

0.145

In [12]:
model.update(labels={"rmse": str(0.145)})

resource name: projects/954208583758/locations/us-central1/models/1372802939441119232

In [ ]:


def register_model_to_vertex(model_name: str, rmse: float, pipeline_root: str) -> Optional[aiplatform.Model]:
    try:
        # Use a filter to find models with the exact display name.
        model_registry = aiplatform.Model.list(filter=f'display_name="{model_name}"')
    except (PermissionDenied, NotFound) as e:
        logging.error(f"Error listing models: {e}")
        return None

    if not model_registry:
        print(f"Registering the first version of {model_name}...")
        try:
            model = aiplatform.Model.upload(
                display_name=model_name,
                artifact_uri=pipeline_root,
                serving_container_image_uri="us-central1-docker.pkg.dev/level-agent-451514-c0/ml-housing-repo/housing-model",
            )
            print(f"Model {model_name} registered as version 1.")

            model.update(labels={"rmse": str(rmse)})
            return model
        except Exception as e:
            logging.error(f"Error registering model: {e}")
            raise

    else:
        # Find existing model (highest version)
        existing_model = None
        highest_version = -1
        for model in model_registry:
            try:
                version_id = int(model.version_id)
                if version_id > highest_version:
                    highest_version = version_id
                    existing_model = model
            except (ValueError, TypeError):
                logging.warning(f"Invalid version ID for model: {model.resource_name}")
                continue

        if not existing_model:
            logging.warning("No valid existing model found.")
            return None

        existing_rmse = float(existing_model.labels.get("rmse", float("inf")))

        if rmse < existing_rmse:
            logging.info(f"Registering a new version of {model_name} with RMSE {rmse} (previous: {existing_rmse})...")
            model = aiplatform.Model.upload(
                display_name=model_name,
                artifact_uri=pipeline_root,
                serving_container_image_uri="us-central1-docker.pkg.dev/level-agent-451514-c0/ml-housing-repo/housing-model",
            )
            logging.info(f"Model {model_name} registered as a new version.")

            model.update(labels={"rmse": str(rmse)})
            return model
        else:
            logging.info(f"New model RMSE {rmse} is not lower than existing RMSE {existing_rmse}. Skipping registration.")
            return None

NameError: name 'aiplatform' is not defined